# Database manipulation

How to edit datasets? Here I will show how to edit the ecoinvent database such that all activities use 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, DE, None) as electricity source instead of 'market for electricity, medium voltage'.

In [39]:
# imports
import brightway2 as bw

# activate project and show databases
bw.projects.set_current("adv-lca-course")
bw.databases

Databases dictionary with 6 object(s):
	bio2
	biosphere3
	cons36
	day1
	day3
	testbiosphere

In [42]:
bw.projects.delete_project("ei36_edited")

'adv-lca-course'

First, we copy the ecoinvent database so that we don't lose the original datasets. The easiest way to do this in brightway is to copy the whole project.

In [43]:
new_project = "ei36_edited"

# Note: brightway will throw an error if a project with the given name already exists. 
# Delete it by using bw.project.delete_project(). After that, you also need to delete 
# the directory from the disk manually!
# bw.projects.delete_project(new_project)

bw.projects.copy_project(new_project)
bw.projects

Brightway2 projects manager with 14 objects:
	Biowood
	Microalgae_1
	ab
	adv-lca-course
	bw25
	default
	dummy
	ei36_edited
	ei_3.5_afk
	ei_3.6_new
	exiobase
	exiobase16
	exiobase_new
	exiobase_test
Use `projects.report()` to get a report on all projects.

We now have a new project "ei36_edited". Let's activate and see which databases are in it.

In [44]:
bw.projects.set_current(new_project)
bw.databases

Databases dictionary with 6 object(s):
	bio2
	biosphere3
	cons36
	day1
	day3
	testbiosphere

As expected, it contains the same databases as the original project. Now lets go about editing the ecoinvent database. First, let's choose the replacing activity. Here, I will use 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, DE, None).

In [45]:
wind = bw.Database("cons36").search("electricity 1-3 wind DE")[0]
wind

'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, DE, None)

Next, we will do the replacement. For this, we loop over all activities in the ecoinvent database. For all activities, we retrieve the technosphere exchanges (i.e. exchanges between activities, not between activity and biosphere). If the exchange's name is "market for electricity, medium voltage" it will be edited. We replace the input (i.e. the producer) of each exchange with our wind energy activity. We will not change any other properties of the exchange (target, amount...). This is a pure "rewiring". **This step will take several minutes**!

In [46]:
for i,act in enumerate(bw.Database("cons36")): 
    for exc in act.technosphere():
        if exc.input["name"] == "market for electricity, medium voltage":
            exc.input = wind
            exc.save()
    if i % 1000 == 0:
        print(f"Checked {i} activities...")

Changed 0 activities...
Changed 1000 activities...
Changed 2000 activities...
Changed 3000 activities...
Changed 4000 activities...
Changed 5000 activities...
Changed 6000 activities...
Changed 7000 activities...
Changed 8000 activities...
Changed 9000 activities...
Changed 10000 activities...
Changed 11000 activities...
Changed 12000 activities...
Changed 13000 activities...
Changed 14000 activities...
Changed 15000 activities...
Changed 16000 activities...


Let's see if the re-wiring worked. Arbitrarily, I pick the activity "limestone production, crushed, washed CH" which I know uses electricity. Here are its technosphere exchanges.

In [48]:
lime_new = bw.Database("cons36").search("limestone production, crushed, washed CH")[0]
list(lime_new.technosphere())

[Exchange: 1.0 kilogram 'market for limestone, unprocessed' (kilogram, CH, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 0.00036 kilowatt hour 'electricity production, hydro, run-of-river' (kilowatt hour, CH, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 0.00036 kilowatt hour 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, DE, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 0.00141 megajoule 'market for heat, central or small-scale, other than natural gas' (megajoule, CH, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 2.07e-08 meter 'market for conveyor belt' (meter, GLO, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 9.11e-06 kilogram 'market for industrial machine, heavy, unspecified' (kilogram, RER, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>]

Indeed, the exchange is now linked to 'electricity production, wind, 1-3MW turbine, onshore'. For comparison, here are the exchanges from the original.

In [53]:
bw.projects.set_current("adv-lca-course")
lime_old = bw.Database("cons36").search("limestone production, crushed, washed CH")[0]
list(lime_old.technosphere())

[Exchange: 1.0 kilogram 'market for limestone, unprocessed' (kilogram, CH, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 0.00036 kilowatt hour 'electricity production, hydro, run-of-river' (kilowatt hour, CH, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 0.00036 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, CH, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 0.00141 megajoule 'market for heat, central or small-scale, other than natural gas' (megajoule, CH, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 2.07e-08 meter 'market for conveyor belt' (meter, GLO, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>,
 Exchange: 9.11e-06 kilogram 'market for industrial machine, heavy, unspecified' (kilogram, RER, None) to 'limestone production, crushed, washed' (kilogram, CH, None)>]

Let's also calculate the GWP 100 of the new and old process for comparison.

In [59]:
method = ('ILCD 2.0 2018 midpoint', 'climate change', 'climate change total')

bw.projects.set_current("adv-lca-course")
bw.calculation_setups["lime_old"] = {'inv': [{lime_old:1}], 'ia': [method]}
mlca_old = bw.MultiLCA("lime_old")

bw.projects.set_current("ei36_edited")
bw.calculation_setups["lime_new"] = {'inv': [{lime_new:1}], 'ia': [method]}
mlca_new = bw.MultiLCA("lime_new")

mlca_old.results, mlca_new.results

(array([[0.0023789]]), array([[0.0022418]]))

As we can see, the GWP100 changed from 0.0024 to 0.0022.